Import required libraries:

In [ ]:
import requests
import json
import os
import pandas as pd

### Set SCRAPE=True in order to download the data from FB

In [ ]:
SCRAPE = True

Define a function to return an empty list if string is not JSON formatted:

In [ ]:
def read_json(path):
    try:
        return json.loads(open(path, "r").read())
    except json.decoder.JSONDecodeError:
        return []

Try to read "posts.json". If it does not exist, create it.

In [ ]:
try:
    posts = read_json("posts.json")
except FileNotFoundError:
    f = open("posts.json", "w")
    f.close
    posts = []

Create a supplemental function to simplify API 

In [ ]:
def get(url_):
    return json.loads(requests.get(url_).text)

def post_exists(post_id):
    return len(list(filter(lambda x: x['id'] == post_id, posts))) > 0

Define gruop ID variables:

In [ ]:
TECHNION_CONFESSIONS_ID = "134517547222780"
TAU_CONFESSIONS_ID = "561380070875128"
IDC_CONFESSIONS_ID = "199527394120566"

Authenticate to Graph API:

In [ ]:
# TOKEN = "EAACEdEose0cBAEIYEOmJ51kfOHPinwL2ychqVpFT0D9ezMjfe1pMKyH0lP2ZA9RuZBH3wQeJ0EfEboHEqDqy7Gq8PSjIcHgNv1lRyC3ZBMk0CdbdzWckHsqNTRatwPZBvkMBfic2P1jwg5vZAZCHKbHdvFfMjjsHGjeVsY7AqEbpWiUBEb0d1zsKJgJwQw3wTWs6ZBb4WSOOQZDZD"

s = "6cc937f2a9dbc9df92600f365c777d1a"
i = "652869818252649"
u = "https://graph.facebook.com/oauth/access_token?client_id={id}&client_secret={secret}&grant_type=client_credentials"

TOKEN = get(u.format(id=i, secret=s))['access_token']

Setup Graph API host and API node:

In [ ]:
HOST = "https://graph.facebook.com/"
API_NODE = "v2.12/"

Declare custom node - in this case, the 'Technion Confessions' group feed:

In [ ]:
TECHNION_CONFESSIONS_NODE = "{}/feed".format(TECHNION_CONFESSIONS_ID)
TAU_CONFESSIONS_NODE = "{}/feed".format(TAU_CONFESSIONS_ID)
IDC_CONFESSIONS_NODE = "{}/feed".format(IDC_CONFESSIONS_ID)

PAGES = [
    (TECHNION_CONFESSIONS_NODE, "TECHNION"),
    (TAU_CONFESSIONS_NODE, "TAU"),
    (IDC_CONFESSIONS_NODE, "IDC"),
]

Build the requested URL - should be the same for the most of the calls:

In [ ]:
def build_url(node):
    return HOST + API_NODE + node + "?access_token={}".format(TOKEN)

## ! Edit the algorithm to add new posts - aka continue from the last post on

The main post scraping function:

In [ ]:
def get_posts(url, origin):
    res = get(url)
    if 'data' in res:
        for post in res['data']:
            if not post_exists(post['id']):
                post['origin'] = origin
                posts.append(post)
    if 'paging' in res:
        if 'next' in res['paging']:
            return get_posts(res['paging']['next'], origin)

If SCAPE, the script will fill "posts.json" with the posts:

In [ ]:
if SCRAPE:
    for node, name in PAGES:
        get_posts(build_url(node), name)
    with open("posts.json", "w") as f:
        f.write(json.dumps(posts))

Show the head of the posts table:

In [ ]:
df = pd.read_json(open("posts.json", "r").read())
df.head()

In [ ]:
df

Get the IDs

Get ID of the posts that the post is replying to

Get the posts likes

Get the comments of the posts

Get the likes for each comment

Get the comments for each comment

Get the name of the commenter (for each comment or reply)

Get the commenter gender